In [1]:
import pandas as pd
import gmaps
import sys
import requests
import json
from sqlalchemy import create_engine
from config import gkey

### Extract CSVs into DataFrames

In [2]:
confirmados_file = "Confirmados.csv"
defunciones_file = "Defunciones.csv"
negativos_file = "Negativos.csv"

### Transform DataFrames

In [3]:
confirmados_data = pd.read_csv(confirmados_file)
confirmados_df = pd.DataFrame(confirmados_data)
confirmados_df = confirmados_df.dropna(how='all')
confirmados_df.insert(3, "status", "confirmed", True) 
confirmados_df.rename(columns = {'nombre':'estado'}, inplace = True)
c1_df = confirmados_df.drop([0])
c1_df.head()

,fips,estado,pais,status,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,...,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,Unnamed: 97,Unnamed: 98
1,1,AGUASCALIENTES,Mexico,confirmed,NaN,NaN,0,0,0,0,...,9,4,9,6,1,1,0,2,NaN,NaN
2,2,BAJA CALIFORNIA,Mexico,confirmed,NaN,NaN,0,0,0,0,...,69,58,34,15,6,23,1,0,NaN,NaN
3,3,BAJA CALIFORNIA SUR,Mexico,confirmed,NaN,NaN,0,0,0,0,...,5,9,5,1,1,0,1,0,NaN,NaN
4,4,CAMPECHE,Mexico,confirmed,NaN,NaN,0,0,0,0,...,2,4,1,5,1,1,2,0,NaN,NaN
5,5,COAHUILA,Mexico,confirmed,NaN,NaN,0,0,0,0,...,13,4,8,1,2,2,12,0,NaN,NaN


In [4]:
#Nacional confirmados
na1 = confirmados_df.loc[confirmados_df['estado'] == "Nacional"]
na1

,fips,estado,pais,status,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,...,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,Unnamed: 97,Unnamed: 98
0,0,Nacional,Mexico,confirmed,NaN,NaN,0,0,0,0,...,686,654,663,419,343,569,265,67,NaN,NaN


In [5]:
negativos_data = pd.read_csv(negativos_file)
negativos_df = pd.DataFrame(negativos_data)
negativos_df = negativos_df.dropna(how='all')
negativos_df.insert(3, "status","negativos", True)
n1_df = negativos_df.drop([0])
n1_df.head()

,fips,estado,pais,status,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,...,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,Unnamed: 97,Unnamed: 98
1,1,AGUASCALIENTES,Mexico,negativos,NaN,NaN,0,1,1,0,...,58,32,20,10,18,23,1,0,NaN,NaN
2,2,BAJA CALIFORNIA,Mexico,negativos,NaN,NaN,2,1,2,0,...,28,33,20,8,5,14,0,0,NaN,NaN
3,3,BAJA CALIFORNIA SUR,Mexico,negativos,NaN,NaN,0,0,0,0,...,36,23,10,3,3,8,9,3,NaN,NaN
4,4,CAMPECHE,Mexico,negativos,NaN,NaN,0,0,0,0,...,7,9,3,5,1,20,2,0,NaN,NaN
5,5,COAHUILA,Mexico,negativos,NaN,NaN,5,3,2,0,...,73,139,98,34,8,44,22,1,NaN,NaN


In [6]:
na2 = negativos_df.loc[negativos_df['estado'] == "Nacional"]
na2

,fips,estado,pais,status,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,...,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,Unnamed: 97,Unnamed: 98
0,0,Nacional,Mexico,negativos,NaN,NaN,56,46,46,15,...,1505,1438,1565,633,520,1261,609,150,NaN,NaN


In [7]:
defunciones_data = pd.read_csv(defunciones_file)
defunciones_df = pd.DataFrame(defunciones_data)
defunciones_df = defunciones_df.dropna(how='all')
defunciones_df.insert(3, "status","defunciones", True)
d1_df = defunciones_df.drop([0])
d1_df.head()

,fips,estado,pais,status,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,...,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,Unnamed: 97,Unnamed: 98
1,1,AGUASCALIENTES,Mexico,defunciones,NaN,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
2,2,BAJA CALIFORNIA,Mexico,defunciones,NaN,NaN,0,0,0,0,...,14,6,8,2,9,5,3,0,NaN,NaN
3,3,BAJA CALIFORNIA SUR,Mexico,defunciones,NaN,NaN,0,0,0,0,...,0,1,1,0,0,1,0,1,NaN,NaN
4,4,CAMPECHE,Mexico,defunciones,NaN,NaN,0,0,0,0,...,0,0,1,0,1,0,0,0,NaN,NaN
5,5,COAHUILA,Mexico,defunciones,NaN,NaN,0,0,0,0,...,1,3,0,0,1,3,1,1,NaN,NaN


In [8]:
na3 = defunciones_df.loc[defunciones_df['estado'] == "Nacional"]
na3

,fips,estado,pais,status,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,...,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,Unnamed: 97,Unnamed: 98
0,0,Nacional,Mexico,defunciones,NaN,NaN,0,0,0,0,...,65,49,56,52,66,58,37,29,NaN,NaN


### Merge all DataFrames

In [9]:
#Merge by state
estados_df = pd.concat([c1_df,n1_df,d1_df], ignore_index=True)
estados_df.fillna('', inplace=True)


In [10]:
#Google API for Lat and Lng
params = {"key": gkey}
for index, row in estados_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    state = row['estado']
    country = row['pais']
    params['address'] = f"{state},{country}"
    state_lat_lng = requests.get(base_url, params=params)
    state_lat_lng = state_lat_lng.json()
    estados_df.loc[index, "Lat"] = float(state_lat_lng["results"][0]["geometry"]["location"]["lat"])
    estados_df.loc[index, "Long"] = float(state_lat_lng["results"][0]["geometry"]["location"]["lng"])
estados_df.head()

,fips,estado,pais,status,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,...,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,Unnamed: 97,Unnamed: 98
0,1,AGUASCALIENTES,Mexico,confirmed,21.8853,-102.292,0,0,0,0,...,9,4,9,6,1,1,0,2,,
1,2,BAJA CALIFORNIA,Mexico,confirmed,30.8406,-115.284,0,0,0,0,...,69,58,34,15,6,23,1,0,,
2,3,BAJA CALIFORNIA SUR,Mexico,confirmed,26.0444,-111.666,0,0,0,0,...,5,9,5,1,1,0,1,0,,
3,4,CAMPECHE,Mexico,confirmed,19.8301,-90.5349,0,0,0,0,...,2,4,1,5,1,1,2,0,,
4,5,COAHUILA,Mexico,confirmed,27.0587,-101.707,0,0,0,0,...,13,4,8,1,2,2,12,0,,


In [11]:
#Merge by nacional
nacional_df = pd.concat([na1,na2,na3], ignore_index=True)
nacional_df.head()

,fips,estado,pais,status,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,...,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,Unnamed: 97,Unnamed: 98
0,0,Nacional,Mexico,confirmed,NaN,NaN,0,0,0,0,...,686,654,663,419,343,569,265,67,NaN,NaN
1,0,Nacional,Mexico,negativos,NaN,NaN,56,46,46,15,...,1505,1438,1565,633,520,1261,609,150,NaN,NaN
2,0,Nacional,Mexico,defunciones,NaN,NaN,0,0,0,0,...,65,49,56,52,66,58,37,29,NaN,NaN


### Create database connection

In [ ]:
rds_connection_string = "postgres:postgres@localhost:5432/Project2"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
# Confirm tables
engine.table_names()

### Load DataFrames into database

In [ ]:
n1_df.to_sql(name='negativos', con=engine, if_exists='append', index=False)

In [ ]:
s1_df.to_sql(name='sospechosos', con=engine, if_exists='append', index=False)

In [ ]:
d1_df.to_sql(name='defunciones', con=engine, if_exists='append', index=False)

In [ ]:
c1_df.to_sql(name='confirmados', con=engine, if_exists='append', index=False)

In [ ]:
estados_df.to_sql(name='estados', con=engine, if_exists='append', index=False)

In [ ]:
nacional_df.to_sql(name='nacional', con=engine, if_exists='append', index=False)

### SQL Tables

In [ ]:
#Just change the table name 
create table confirmados/sospechosos/negativos/defunciones/estados/nacional
(
fips varchar(100)
estado varchar(100),
pais varchar(100),
status varchar(100),
Lat int,
Long int,
"January_22" string,
"January_23" string,
"January_24" string,
"January_25" string,
"January_26" string,
"January_27" string,
"January_28" string,
"January_29" string,
"January_30" string,
"January_31" string,
"Feburary_1" string,
"Feburary_2" string,
"Feburary_3" string,
"Feburary_4" string,
"Feburary_5" string,
"Feburary_6" string,
"Feburary_7" string,
"Feburary_8" string,
"Feburary_9" string,
"Feburary_10" string,
"Feburary_11" string,
"Feburary_12" string,
"Feburary_13" string,
"Feburary_14" string,
"Feburary_15" string,
"Feburary_16" string,
"Feburary_17" string,
"Feburary_18" string,
"Feburary_19" string,
"Feburary_20" string,
"Feburary_21" string,
"Feburary_22" string,
"Feburary_23" string,
"Feburary_24" string,
"Feburary_25" string,
"Feburary_26" string,
"Feburary_27" string,
"Feburary_28" string,
"Feburary_29" string,
"March_1" string,
"March_2" string,
"March_3" string,
"March_4" string,
"March_5" string,
"March_6" string,
"March_7" string,
"March_8" string,
"March_9" string,
"March_10" string,
"March_11" string,
"March_12" string,
"March_13" string,
"March_14" string,
"March_15" string,
"March_16" string,
"March_17" string,
"March_18" string,
"March_19" string,
"March_20" string,
"March_21" string,
"March_22" string,
"March_23" string,
"March_24" string,
"March_25" string,
"March_26" string,
"March_27" string,
"March_28" string,
"March_29" string,
"March_30" string,
"March_31" string,
"April_1" string,
"April_2" string,
"April_3" string,
"April_4" string,
"April_5" string,
"April_6" string,
"April_7" string,
"April_8" string,
"April_9" string,
"April_10" string,
"April_11" string,
"April_12" string,
"April_13" string,
"April_14" string,
"April_15" string,
"April_16" string,
"April_17" string,
"April_18" string,
"April_19" string,
"April_20" string,
"April_21" string,
"April_22" string
)